# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [1]:
# Imports
import os
if('notebooks' in os.getcwd()):
    os.chdir('..')
import json
import pandas as pd
from utils.enums import AirFiles, CO2Files
from utils.load_data import load_data_with_event_matching
from utils.timestamps import read_unix, read_j2000, j2000_to_unix
import numpy as np


In [2]:
# Open file index JSON for reading
file_index = pd.read_json(os.path.join(os.getcwd(), 'temp/file_index.json'))
file_index = file_index.sort_index()

In [3]:
# Example how to retrieve file names for a participant
participant_to_retrieve = 12
air_mask_file = file_index[0][participant_to_retrieve-1][AirFiles.MASK.value]
air_event_file = file_index[0][participant_to_retrieve-1][AirFiles.EVENT.value]

In [4]:
participant_df = pd.DataFrame()
if os.path.exists(air_mask_file):
    if os.path.exists(air_mask_file):
        print('Loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
        participant_df = load_data_with_event_matching(air_mask_file, True, air_event_file)
        print('Finished loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
        participant_df
    else:
        print("Event file missing for participant: " + str(participant_to_retrieve))
else:
    print("Mask file missing for participant: " + str(participant_to_retrieve))





Loading data for participant: 12_alex_b
Finished loading data for participant: 12_alex_b


D:\co2-study\utils\load_data.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Event'][event_row_index] = events[i]


In [5]:
# Load eye tracking data
participant_eye_df = pd.DataFrame()
air_eyetracking_file = file_index[0][participant_to_retrieve-1][AirFiles.EYE.value]

if os.path.exists(air_eyetracking_file):
    print('Loading eye data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
    participant_eye_df = pd.read_csv(air_eyetracking_file)
    print('Finished loading eye data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
    # First couple frames of eye tracking data are weird. Drop them.
    participant_eye_df = participant_eye_df.drop(index=range(10)).reset_index(drop=True)
else:
    print("Eye file missing for participant: " + str(participant_to_retrieve))

Loading eye data for participant: 12_alex_b
Finished loading eye data for participant: 12_alex_b


In [6]:
# Add eye tracking data

# Potential improvements: 
# 1. ignore first 10 eye tracking values as they are a bit weird - DONE
# 2. remove Magnetometer columns

if participant_df.empty or participant_eye_df.empty:
    print('Cant synchronise eye tracking data. One of the files is empty')
else:
    #Some eye tracking files (participants 1-9) have empty 'TimestampUnix column'. Use their 'TimestampJ2000' to calculate unix column
    if np.mean(participant_eye_df['TimestampUnix'].values)==0:
        print('TimestampUnix column for eye data is empty. Converting J2000 timestamps to Unix')
        j2000_timestamps = participant_eye_df['TimestampJ2000']
        unix_timestamps = []
        for j2000_timestamp in j2000_timestamps:
            unix_timestamps.append(j2000_to_unix(j2000_timestamp))
        participant_eye_df['TimestampUnix']=unix_timestamps
        
    participant_eye_df['TimestampUnix'] = participant_eye_df['TimestampUnix']/1000
        
    # TODO: SYNCHRONISE EYE DATA WITH MASK DF
    print('Synchronising mask and eye data files.')
    columns = ['VerboseData.Right.PupilDiameterMm', 'VerboseData.Left.PupilDiameterMm']
    synced_eye_df = pd.DataFrame(np.nan, index=participant_df.index, columns=columns)
    for index, eye_data_row in participant_eye_df.iterrows():
        timestamp_to_find = eye_data_row['TimestampUnix']
        event_row_index = participant_df['unix_timestamp'].searchsorted(timestamp_to_find)
        synced_eye_df.loc[event_row_index] = eye_data_row[columns]
    # eye data file sometimes runs for a few seconds longer than the mask data. 
    # Sync function above contiounsly overwrites last row of data with those eye data frames. Set the last row back to 0 for forward propagation later.     
    synced_eye_df.loc[len(synced_eye_df)-1]=np.nan
    participant_df[columns] = synced_eye_df
    print('Forward filling eye tracking data')
    participant_df[columns] = participant_df[columns].ffill()
    print('Finished synchronising eye tracking with mask data')
    

Synchronising mask and eye data files.
Forward filling eye tracking data
Finished synchronising eye tracking with mask data


In [ ]:
participant_df

In [ ]:
# Add biopac data

In [ ]:
# Convert Unix
print(read_unix(1651081973.94119))

In [ ]:
# Convert J2000
print(read_j2000(704397173515))